In [1]:
#This code run only with Gensim 3.8.3 so we remove 4.0 and then we install gensim 3.8.3

#!pip uninstall gensim  

#!pip install gensim==3.8.3

In [1]:
#import

from gensim import models
import gensim
import numpy as np
from statistics import mean
import nltk
from nltk.corpus import stopwords
import re

In [2]:
#path we are going to use 

chosenidioms= "C:/Users/Giulia Santoro/Desktop/cl/esame/chosen_idioms.txt"
fulldataset= "C:/Users/Giulia Santoro/Desktop/cl/esame/dataset completo.txt"
lphrases= "C:/Users/Giulia Santoro/Desktop/cl/esame/literal_phrases.txt"
nlphrases= "C:/Users/Giulia Santoro/Desktop/cl/esame/non_literal_phrases.txt"


In [3]:
#we restrict the google vectors passing our dataset to this function

def restrict_w2v(w2v, restricted_word_set):
    new_vectors = []
    new_vocab = {}
    new_index2entity = []
    new_vectors_norm = []

    for i in range(len(w2v.vocab)):
        word = w2v.index2entity[i]
        vec = w2v.vectors[i]
        vocab = w2v.vocab[word]
        if word in restricted_word_set:
            vocab.index = len(new_index2entity)
            new_index2entity.append(word)
            new_vocab[word] = vocab
            new_vectors.append(vec)

    w2v.vocab = new_vocab
    w2v.vectors = np.array(new_vectors)
    w2v.index2entity = np.array(new_index2entity)
    w2v.index2word = np.array(new_index2entity)
    
#import google vectors 

model = models.KeyedVectors.load_word2vec_format(
    'C:/Users/Giulia Santoro/Desktop/cl/esame/GoogleNews-vectors-negative300.bin', binary=True)

In [5]:
# we create my own personal vectors starting from my cleaned and tokenized dataset

corpus = open(fulldataset, encoding="utf8") 

astring = corpus.read()

tokens = nltk.word_tokenize(astring)

words = [word.lower() for word in tokens if word.isalpha()] #eliminate punctuation

#we create our personal stopword set to add to the one nltk provide

ourset={"the","neither", "shall", "no", "yes", "that", "a","in","may","would","can","could","rather","got"} 
stop_words = set(stopwords.words('english'))|ourset

vocab = list(set([word for word in words if not word in stop_words])) #we create the vocab to pass to the function restrict

#print(vocab)

restricted = restrict_w2v(model, vocab)

['bed', 'might', 'rise', 'put', 'socks', 'go', 'cemetery', 'dresses', 'development', 'although', 'behaving', 'aspect', 'exam', 'apathy', 'physiotherapist', 'public', 'buttercream', 'shops', 'try', 'wonderful', 'weeks', 'father', 'spring', 'account', 'paid', 'temporary', 'never', 'stubble', 'suffered', 'bar', 'life', 'statistic', 'smelled', 'thirty', 'came', 'looked', 'room', 'arrested', 'martha', 'car', 'latest', 'law', 'birthday', 'pointing', 'creatures', 'distinction', 'vegetarian', 'less', 'red', 'man', 'pants', 'wind', 'easy', 'future', 'slower', 'name', 'origins', 'government', 'merely', 'detail', 'social', 'fishing', 'looks', 'sands', 'head', 'light', 'nowhere', 'dead', 'tonight', 'feet', 'doctor', 'number', 'buy', 'obstacle', 'fog', 'fluttering', 'codes', 'mechanic', 'parental', 'views', 'hours', 'continue', 'alive', 'become', 'fifteen', 'breeding', 'museum', 'ink', 'lay', 'well', 'christmas', 'financial', 'freebie', 'melted', 'match', 'time', 'eat', 'beautiful', 'jumpsuit', 'ti

In [7]:
#this function create a list of list of words (every sentence is a list of tokenized words and is stored in a bigger list)

def listcreator(filename, stop_words={}):#stopword is empty to give the opportunity to decide to pass our stopword set or not
    if len(stop_words) == 0:
        stop_words = set(nltk.corpus.stopwords.words('english'))
    #print(stop_words)
    
    with open(filename, "r") as handlefile:
        flines=handlefile.readlines()
        group= []
        filtered= []
   
        for line in flines:
            group.append(nltk.word_tokenize(line))

        for line in group:
            line = [each_string.lower() for each_string in line]
            ## the double loop is needed to make to remove steps, punct and stopwords
            for word in line:
                for word in line:
                    if not word.isalpha():
                        line.remove(word)
                    if word in stop_words:
                        line.remove(word)
            #we discard the square brackets
            if line != []:
                filtered.append(line)
       
        return filtered

In [8]:
#we apply the function to our split dataset (nonliteral e literal)

literal_list = listcreator(lphrases, stop_words)
nonliteral_list = listcreator(nlphrases, stop_words)

#print("Literal tokenized list of list:\n")
#print(literal_list)
#print("\nNon-literal tokenized list of list:\n")
#print(nonliteral_list)

Literal tokenized list of list:

[['vegetarian', 'people', 'eat', 'fish', 'fowl'], ['impossible', 'fish', 'fowl', 'raised', 'breeding', 'farm'], ['shop', 'sell', 'fish', 'fowl'], ['essay', 'results', 'conclusions', 'come', 'next', 'nothing'], ['chair', 'next', 'nothing'], ['build', 'new', 'shopping', 'centre', 'next', 'nothing'], ['exam', 'precisely', 'started', 'nine', 'five', 'finished', 'two', 'hours'], ['yesterday', 'match', 'manchester', 'united', 'beated', 'real', 'madrid', 'nine', 'five'], ['number', 'participants', 'drastically', 'reduced', 'nine', 'five'], ['year', 'local', 'farmes', 'spring', 'chicken', 'sell'], ['man', 'opened', 'fridge', 'spring', 'chicken', 'left', 'dinner'], ['fact', 'spring', 'chicken', 'tastier', 'expensive', 'veal', 'months'], ['martha', 'know', 'grandfather', 'dead', 'buried', 'cemetery', 'somewhere', 'india'], ['dogs', 'dead', 'buried', 'garden', 'still', 'complain', 'daily'], ['sam', 'father', 'stil', 'alive', 'dead', 'buried', 'fifty', 'years'], ['

In [9]:
#we calculate the cosine similarity between the words in every sentence. We could have done it putting all the value in a matrix
#with all the words and then retrieve the values we needed but it was a waste of computational power, since we want to do the 
#cosine similarity of the words in a given line and not of all the words. 

#we also could have calculated the cosine similarity between all the words in a sentence and put it in a matrix but we did not want
#to calculate the value repetitions typical of a matrix (ex: of the line I like fish you calculate I and like but also like e I).
#it was no problem however as the cosine similarity should have be averaged after this step

#we opted for scanning the list of list with indexes, and then calculate the cosine similarity incrementally 
#and avoiding repetitions

def cossimll(mylist):

    cosinell= []
    cosinel= []
    for sentence in mylist:
        myarray= np.array(sentence)
        #print("\n***** New sentence to analize: ******\n" + str(myarray))
        x=1
        for i in range(len(sentence)):
            #print("  <------->")
            #print("Word from compare:->  " + myarray[i])
            sourcetocompare = myarray[i]
            j=x
            for k in range(len(myarray)-j):
                #print("Word  to  compare:->  " + sentence[j])
                desttocompare = sentence[j]
            
                print("source:-> " + sourcetocompare + "     dest:-> " + desttocompare +"\n")
                try:
                    cosine_similarity = model.similarity(myarray[i], sentence[j])
                    #print("cosine similarity between "+ sourcetocompare + " and " + desttocompare + " is:")
                    #print(cosine_similarity)
                    #print("\n")
                    cosinel.append(cosine_similarity)
                except:
                    pass
                j=j+1
            x=x+1
        cosinell.append(cosinel)
        cosinel=[]    
        
    return cosinell      

literalcossim= cossimll(literal_list)
nliteralcossim= cossimll(nonliteral_list)

#print("\nCosine similarity for each word of each literal sentence:\n")
#print(literalcossim)
#print("\nCosine similarity for each word of each non-literal sentence:\n")
#print(nliteralcossim)




***** New sentence to analize: ******
['vegetarian' 'people' 'eat' 'fish' 'fowl']
  <------->
Word from compare:->  vegetarian
Word  to  compare:->  people
source:-> vegetarian     dest:-> people

cosine similarity between vegetarian and people is:
0.13274178


Word  to  compare:->  eat
source:-> vegetarian     dest:-> eat

cosine similarity between vegetarian and eat is:
0.47242546


Word  to  compare:->  fish
source:-> vegetarian     dest:-> fish

cosine similarity between vegetarian and fish is:
0.22092974


Word  to  compare:->  fowl
source:-> vegetarian     dest:-> fowl

cosine similarity between vegetarian and fowl is:
0.17157438


  <------->
Word from compare:->  people
Word  to  compare:->  eat
source:-> people     dest:-> eat

cosine similarity between people and eat is:
0.23179178


Word  to  compare:->  fish
source:-> people     dest:-> fish

cosine similarity between people and fish is:
0.20982423


Word  to  compare:->  fowl
source:-> people     dest:-> fowl

cosine simi

cosine similarity between red and black is:
0.4841255


Word  to  compare:->  one
source:-> red     dest:-> one

cosine similarity between red and one is:
0.17868148


  <------->
Word from compare:->  ink
Word  to  compare:->  black
source:-> ink     dest:-> black

cosine similarity between ink and black is:
0.20170575


Word  to  compare:->  one
source:-> ink     dest:-> one

cosine similarity between ink and one is:
0.053097233


  <------->
Word from compare:->  black
Word  to  compare:->  one
source:-> black     dest:-> one

cosine similarity between black and one is:
0.16257358


  <------->
Word from compare:->  one

***** New sentence to analize: ******
['time' 'teacher' 'made' 'corrections' 'blue' 'instead' 'using' 'red'
 'ink']
  <------->
Word from compare:->  time
Word  to  compare:->  teacher
source:-> time     dest:-> teacher

cosine similarity between time and teacher is:
0.113066494


Word  to  compare:->  made
source:-> time     dest:-> made

cosine similarity between 

  <------->
Word from compare:->  head

***** New sentence to analize: ******
['went' 'doctor' 'terrible' 'pain' 'neck']
  <------->
Word from compare:->  went
Word  to  compare:->  doctor
source:-> went     dest:-> doctor

cosine similarity between went and doctor is:
0.12333202


Word  to  compare:->  terrible
source:-> went     dest:-> terrible

cosine similarity between went and terrible is:
0.1973739


Word  to  compare:->  pain
source:-> went     dest:-> pain

cosine similarity between went and pain is:
0.059722774


Word  to  compare:->  neck
source:-> went     dest:-> neck

cosine similarity between went and neck is:
0.091041625


  <------->
Word from compare:->  doctor
Word  to  compare:->  terrible
source:-> doctor     dest:-> terrible

cosine similarity between doctor and terrible is:
0.13595393


Word  to  compare:->  pain
source:-> doctor     dest:-> pain

cosine similarity between doctor and pain is:
0.2944064


Word  to  compare:->  neck
source:-> doctor     dest:-> nec

cosine similarity between pants and old is:
0.10596454


Word  to  compare:->  hat
source:-> pants     dest:-> hat

cosine similarity between pants and hat is:
0.47957245


  <------->
Word from compare:->  torn
Word  to  compare:->  old
source:-> torn     dest:-> old

cosine similarity between torn and old is:
0.19853592


Word  to  compare:->  hat
source:-> torn     dest:-> hat

cosine similarity between torn and hat is:
0.04743179


  <------->
Word from compare:->  old
Word  to  compare:->  hat
source:-> old     dest:-> hat

cosine similarity between old and hat is:
0.086027294


  <------->
Word from compare:->  hat

***** New sentence to analize: ******
['anne' 'wearing' 'nice' 'dress' 'old' 'hat' 'mother']
  <------->
Word from compare:->  anne
Word  to  compare:->  wearing
source:-> anne     dest:-> wearing

cosine similarity between anne and wearing is:
0.10200243


Word  to  compare:->  nice
source:-> anne     dest:-> nice

cosine similarity between anne and nice is:
0.120663


cosine similarity between bit and spring is:
0.18618053


Word  to  compare:->  chicken
source:-> bit     dest:-> chicken

cosine similarity between bit and chicken is:
0.111298904


  <------->
Word from compare:->  bright
Word  to  compare:->  young
source:-> bright     dest:-> young

cosine similarity between bright and young is:
0.34956127


Word  to  compare:->  thought
source:-> bright     dest:-> thought

cosine similarity between bright and thought is:
0.103999674


Word  to  compare:->  hell
source:-> bright     dest:-> hell

cosine similarity between bright and hell is:
0.049078505


Word  to  compare:->  spring
source:-> bright     dest:-> spring

cosine similarity between bright and spring is:
0.12518144


Word  to  compare:->  chicken
source:-> bright     dest:-> chicken

cosine similarity between bright and chicken is:
0.06681351


  <------->
Word from compare:->  young
Word  to  compare:->  thought
source:-> young     dest:-> thought

cosine similarity between young an

cosine similarity between never and spent is:
-0.023470141


Word  to  compare:->  week
source:-> never     dest:-> week

cosine similarity between never and week is:
0.19183384


Word  to  compare:->  milksopping
source:-> never     dest:-> milksopping

Word  to  compare:->  around
source:-> never     dest:-> around

cosine similarity between never and around is:
0.09530037


Word  to  compare:->  godforsaken
source:-> never     dest:-> godforsaken

cosine similarity between never and godforsaken is:
0.15361708


Word  to  compare:->  backwater
source:-> never     dest:-> backwater

cosine similarity between never and backwater is:
0.13251795


Word  to  compare:->  account
source:-> never     dest:-> account

cosine similarity between never and account is:
0.10529891


Word  to  compare:->  one
source:-> never     dest:-> one

cosine similarity between never and one is:
0.33528668


Word  to  compare:->  little
source:-> never     dest:-> little

cosine similarity between never and l

cosine similarity between years and shops is:
0.052064344


Word  to  compare:->  left
source:-> years     dest:-> left

cosine similarity between years and left is:
0.06048111


Word  to  compare:->  private
source:-> years     dest:-> private

cosine similarity between years and private is:
0.101808555


Word  to  compare:->  sector
source:-> years     dest:-> sector

cosine similarity between years and sector is:
0.1073041


  <------->
Word from compare:->  like
Word  to  compare:->  restaurants
source:-> like     dest:-> restaurants

cosine similarity between like and restaurants is:
0.07044849


Word  to  compare:->  shops
source:-> like     dest:-> shops

cosine similarity between like and shops is:
0.11673035


Word  to  compare:->  left
source:-> like     dest:-> left

cosine similarity between like and left is:
0.09813454


Word  to  compare:->  private
source:-> like     dest:-> private

cosine similarity between like and private is:
0.12552038


Word  to  compare:->  sector

  <------->
Word from compare:->  fog
Word  to  compare:->  detail
source:-> fog     dest:-> detail

cosine similarity between fog and detail is:
0.09020876


  <------->
Word from compare:->  detail

***** New sentence to analize: ******
['long' 'course' 'examine' 'every' 'aspect' 'technique' 'every' 'club'
 'bag']
  <------->
Word from compare:->  long
Word  to  compare:->  course
source:-> long     dest:-> course

cosine similarity between long and course is:
0.21527077


Word  to  compare:->  examine
source:-> long     dest:-> examine

cosine similarity between long and examine is:
0.10095492


Word  to  compare:->  every
source:-> long     dest:-> every

cosine similarity between long and every is:
0.15181184


Word  to  compare:->  aspect
source:-> long     dest:-> aspect

cosine similarity between long and aspect is:
0.07207836


Word  to  compare:->  technique
source:-> long     dest:-> technique

cosine similarity between long and technique is:
0.06486733


Word  to  compare:-

cosine similarity between green and first is:
0.040669218


Word  to  compare:->  meeting
source:-> green     dest:-> meeting

cosine similarity between green and meeting is:
0.05911964


Word  to  compare:->  teachers
source:-> green     dest:-> teachers

cosine similarity between green and teachers is:
0.036466274


  <------->
Word from compare:->  light
Word  to  compare:->  head
source:-> light     dest:-> head

cosine similarity between light and head is:
-0.00857801


Word  to  compare:->  discuss
source:-> light     dest:-> discuss

cosine similarity between light and discuss is:
0.10708159


Word  to  compare:->  happen
source:-> light     dest:-> happen

cosine similarity between light and happen is:
0.09656595


Word  to  compare:->  first
source:-> light     dest:-> first

cosine similarity between light and first is:
0.112702444


Word  to  compare:->  meeting
source:-> light     dest:-> meeting

cosine similarity between light and meeting is:
0.084148735


Word  to  compa

Word  to  compare:->  severe
source:-> flag     dest:-> severe

cosine similarity between flag and severe is:
0.075200826


  <------->
Word from compare:->  indicates
Word  to  compare:->  reason
source:-> indicates     dest:-> reason

cosine similarity between indicates and reason is:
0.25552568


Word  to  compare:->  cease
source:-> indicates     dest:-> cease

cosine similarity between indicates and cease is:
0.04565707


Word  to  compare:->  back
source:-> indicates     dest:-> back

cosine similarity between indicates and back is:
0.04407115


Word  to  compare:->  away
source:-> indicates     dest:-> away

cosine similarity between indicates and away is:
0.052416243


Word  to  compare:->  relationship
source:-> indicates     dest:-> relationship

cosine similarity between indicates and relationship is:
0.116105154


Word  to  compare:->  yellow
source:-> indicates     dest:-> yellow

cosine similarity between indicates and yellow is:
0.09697231


Word  to  compare:->  flags
s

In [10]:
#we do the average of every sentence in each list

def listaverage(list_):

    listresults= []
    for l in list_:
        listresults.append(mean(l))
    
    return listresults   

literalaverage=listaverage(literalcossim)
nliteralaverage= listaverage(nliteralcossim)


#print("Average of the words' cosine similarity of each literal sentence:\n")
#print(literalaverage)
#print("\nAverage of the words' cosine similarity of each non-literal sentence:\n")
#print(nliteralaverage)

Average of the words' cosine similarity of each literal sentence:

[0.26175642, 0.16188204, 0.16392808, 0.114848785, 0.05082169, 0.14706454, 0.2170066, 0.113525845, 0.2183144, 0.13564456, 0.121983744, 0.1422521, 0.16023102, 0.13034289, 0.16788024, 0.13542785, 0.1583215, 0.12195687, 0.17333563, 0.11303363, 0.11796447, 0.121142834, 0.11979605, 0.11976593, 0.096813425, 0.112568416, 0.15524434, 0.12430707, 0.15882099, 0.16161332, 0.13547242, 0.17648514, 0.18144014, 0.11935888, 0.119875126, 0.11581683, 0.13383827, 0.15991057, 0.19465512, 0.10388838, 0.10954124, 0.17673637, 0.15902078, 0.19608767, 0.13667646, 0.20346339, 0.22976157, 0.19766352, 0.19135453, 0.1985814, 0.16546677, 0.17416623, 0.09962866, 0.17660305, 0.32320583, 0.19974767, 0.36005282, 0.1552618, 0.18699133, 0.093861505]

Average of the words' cosine similarity of each non-literal sentence:

[0.08225548, 0.14305694, 0.14665425, 0.099747576, 0.1277885, 0.27109703, 0.28024656, 0.21681672, 0.15243544, 0.037917703, 0.15180966, 0.13

In [11]:
#We check if all the sentences are in the same number. We need this to do the next step: handle the list of averages knowing how
#many sentences cointain a specific idiom

def countidioms(idiomf, file):
    count= 0
    listcount=[]
    with open(idiomf, "r") as idiomf_:
        strings= idiomf_.readlines()
        idiomlist= [idiom.strip() for idiom in strings]

        with open(file, "r") as file_:
            strings2=file_.readlines()
            for idiom in idiomlist:
                for string in strings2:
                    if re.search(idiom, string, re.IGNORECASE):
                        count=count+1
                        
                listcount.append(count)
                count=0            
    return listcount

lcount= countidioms(chosenidioms, lphrases)
nlcount= countidioms(chosenidioms, nlphrases)

#print("count for literal sentences for each idiom:\n")
#print(lcount)
#print("\ncount for non-literal sentences for each idiom:\n")
#print(nlcount)

count for literal sentences for each idiom:

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

count for non-literal sentences for each idiom:

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [12]:
#we create a function that does the average of the average results of each sentence with a specific idiom

def averageofav (count, mylist):
    meanlist=[]
    j=0
    k=0
    for i in count:
        k=k+i
        meanlist.append(mean(mylist[j:k]))
        j=j+i
    return meanlist

lvaluelist= averageofav(lcount, literalaverage)
nlvaluelist= averageofav(nlcount, nliteralaverage)

#print("Average of literal sentences for each idiom (20):\n")
#print(lvaluelist)
#print("\nAverage of non-literal sentences for each idiom (20):\n")
#print(nlvaluelist)

Average of literal sentences for each idiom (20):

[0.19585551, 0.10424501, 0.18294895, 0.13329346, 0.15281805, 0.13856874, 0.1347779, 0.12023494, 0.12154206, 0.14824712, 0.1644659, 0.11835028, 0.16280133, 0.13005532, 0.1639283, 0.21029615, 0.18513423, 0.15013264, 0.29433542, 0.14537154]

Average of non-literal sentences for each idiom (20):

[0.12398889, 0.16621104, 0.21649957, 0.10802073, 0.135721, 0.10127969, 0.11439883, 0.11119395, 0.13278304, 0.16394065, 0.10179069, 0.14707595, 0.101539254, 0.13384219, 0.13339825, 0.10347879, 0.10079224, 0.11249034, 0.12441238, 0.097916104]


In [13]:
def createdict(idioms, valuelist):
    
    with open(idioms, "r") as filesource:
        strings= filesource.readlines()
    
        idiomlist= [idiom.strip() for idiom in strings]
        #print(idiomlist) 
    
        dict_= dict(zip(idiomlist,valuelist))
        
        return dict_
    
ldict= createdict(chosenidioms, lvaluelist)
nldict= createdict(chosenidioms, nlvaluelist)

#print("Dictionary of idioms as keys and average of literal sentences for each idiom as value:\n")
#print(ldict)
#print("\nDictionary of idioms as keys and average of non-literal sentences for each idiom as value:\n")
#print(nldict)

Dictionary of idioms as keys and average of literal sentences for each idiom as value:

{'neither fish nor fowl': 0.19585551, 'next to nothing': 0.10424501, 'nine to five': 0.18294895, 'no spring chicken': 0.13329346, 'dead and buried': 0.15281805, 'red ink': 0.13856874, 'bag of bones': 0.1347779, 'behind bars': 0.12023494, 'bells and whistles': 0.12154206, 'let sleeping dogs lie': 0.14824712, 'pain in the neck': 0.1644659, 'in a fog': 0.11835028, 'in the bag': 0.16280133, 'in the same boat': 0.13005532, 'itchy feet': 0.1639283, 'green light': 0.21029615, 'old hat': 0.18513423, 'fly on the wall': 0.15013264, 'icing on the cake': 0.29433542, 'red flag': 0.14537154}

Dictionary of idioms as keys and average of non-literal sentences for each idiom as value:

{'neither fish nor fowl': 0.12398889, 'next to nothing': 0.16621104, 'nine to five': 0.21649957, 'no spring chicken': 0.10802073, 'dead and buried': 0.135721, 'red ink': 0.10127969, 'bag of bones': 0.11439883, 'behind bars': 0.1111939

In [18]:
#as we start from the idea that literal sentences should be more congruent than the non-literal counterparts aka nearer to 1 
#we compare the values of the two dicts and we put the result in two counters

with open(chosenidioms, "r") as filesource:
    strings= filesource.readlines()
    
    idiomlist= [idiom.strip() for idiom in strings]
    #print(idiomlist) 
    
    neilist=[]
    expected= 0
    notexpected= 0 
    for idiom in idiomlist:
        try:
            if ldict[idiom] > nldict[idiom]:
                expected+= 1
            
            else:
                notexpected+=1
                nei= idiom
                neilist.append(nei)
    
        except: 
            pass
        
print("Expected results: literal sentences are more congruent than non-literal:\n")
print(expected)
print("\nNot expected results: non-literal sentences are more congruent than literal:\n")
print(notexpected)
print("\nThese are the idioms which non-literal sentences were more congruent than their literal counterpart:\n")
print(neilist)

Expected results: literal sentences are more congruent than non-literal:

14

Not expected results: non-literal sentences are more congruent than literal:

6

These are the idioms which non-literal sentences were more congruent than their literal counterpart

['next to nothing', 'nine to five', 'bells and whistles', 'let sleeping dogs lie', 'in a fog', 'in the same boat']
